# Randomisation

This tab allows to have a look at thje balance between groups. First, you can select the metadata that will be monitored in this tab (list of selected metadata is saved in the config file). Second, the selected metadata are listed in a table and size of subgroups are given. An indicator of balance is given for each metadata (*green*: balanced, *orange*: risk of unbalanced, and *red*: unbalanced), see explanation about the method used at the end of this page.

In [1]:
from functions.config import Config

config = Config()
config.read()    

In [7]:
import ipywidgets as widgets

if config.is_key('list_metadata'):
    list_metadata = config.get_value('list_metadata')
else:
    list_metadata = config.extract_config_values('list_metadata')
    config.write()
    
if config.is_key('monitored_metadata'):
    list_selected = config.get_value('monitored_metadata')
else:
    list_selected = list_metadata[:1]

all_metadata = widgets.Select(
    options=list_metadata,
    value=list_metadata[0],
    description='Metadata:',
    disabled=False
)

selected_metadata = widgets.Select(
    options=list_selected,
    value=list_selected[0],
    description='Monitored:',
    disabled=False
)

def on_button_add_clicked(var):
    list_selected.append(all_metadata.value)
    selected_metadata.options = list_selected

def on_button_remove_clicked(var):
    list_selected.remove(selected_metadata.value)
    selected_metadata.options = list_selected

def on_button_save_clicked(var):
    config.set_value(list_selected, 'monitored_metadata', section='METADATA')
    config.write()
    
button_add = widgets.Button(description='Add', command=on_button_add_clicked)
button_remove = widgets.Button(description='Remove', command=on_button_remove_clicked)
button_save = widgets.Button(description='Save', command=on_button_save_clicked)

button_add.on_click(on_button_add_clicked)
button_remove.on_click(on_button_remove_clicked)
button_save.on_click(on_button_save_clicked)

widgets.HBox([all_metadata, widgets.VBox([button_add, button_remove, button_save]), selected_metadata])

In [8]:
config.get_value('monitored_metadata')

['Metadata 0', 'Metadata 1', 'Metadata 4']